# From Decision Trees to Random Forests

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a **boostrap** sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# from sklearn.datasets import make_classification
# X, y = make_classification(n_samples=150, n_features=4, n_informative=4, n_redundant=0, n_clusters_per_class=1, flip_y=0.2, class_sep=0.05)
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [3]:
def bootstrap_idx(X):
    # Draw from the dataset with the exact same number of points with replacement.
    indices = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    return indices

In [4]:
bootstrap_idx(X_train)

array([ 23,  73,  15,   9,  73,  93,  32,  78,  44,  44,  85,  86, 101,
        98,  34,  36,  79,   6,  80,  37,  55, 105,  81,  97,  90, 104,
        44,  55,  92,  62,  72,  99,   4,  91,  20,   6,  75, 106,   6,
        97,   2,  87,  70,   1,  79,  41,  38,  22,  47,  99,  39,  41,
       102,   1,  32,   2,  11,   9,  36,  91,  68,  35,  15,  29,  17,
        70,   9,  43,  10,  83,  61,  11,  40,  99,  14,  74,  67,  32,
        36,  27,   3,  73,  76,  85,  92,  99,   9,   7,  26,  63,  72,
        89,  75,  80, 100,  44,  49,  35,  33,  65,  13,  96,  22,  89,
        51, 109,  59, 105,  29,  36,  63,  11])

In [6]:
from collections import Counter

Counter(bootstrap_idx(X_train)).most_common(10)

[(15, 4),
 (95, 3),
 (43, 3),
 (51, 3),
 (77, 3),
 (3, 3),
 (52, 3),
 (82, 3),
 (78, 3),
 (97, 3)]

In [7]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [8]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [9]:
print(f"Classes distribution in the original data: {Counter(y_train)}")
print(f"Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}")

Classes distribution in the original data: Counter({0: 38, 1: 37, 2: 37})
Classes distribution in the bootstrap: Counter({0: 39, 2: 38, 1: 35})


<div class="alert alert-success">
    <b>EXERCISE: Create a bagging classifier</b>:<br>
    <br>
    A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.
     <ul>
      <li>
      Create several <code>DecisionTreeClassifier</code> and store them in a Python list;
      </li>
      <li>
      Loop over these trees and <code>fit</code> them by generating a bootstrap sample using <code>bootstrap_sample</code> function;
      </li>
      <li>
      To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the <code>predict</code> method. Aggregate all predictions in a NumPy array;
      </li>
      <li>
      Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
      </li>
      <li>
      Finally, check the accuracy of your model.
      </li>
    </ul>
</div>

In [12]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score

ntrees = 5
max_depth = 3

tree_list = []
ypred_list = []
for i in range(ntrees):
    X_train_i, y_train_i = bootstrap_sample(X_train, y_train)
    treei = DecisionTreeClassifier(max_depth=max_depth)
    treei.fit(X_train_i, y_train_i)
    ypred_list.append(treei.predict(X_test))
    tree_list.append(treei)

ypred, _ = mode(np.array(ypred_list), axis=0, keepdims=False)
print(accuracy_score(y_test, ypred))

0.9473684210526315


> Bootstraping regularizes the overfitting on the subtrees.

<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own bagging classifier, use a <code>BaggingClassifier</code> from scikit-learn to fit the above data.
</div>

In [ ]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=3), n_estimators=5
)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

1.0


## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

<div class="alert alert-success">
    <b>EXERCISE: Create a random forest classifier</b>:
    <br>
    Use your previous code which was generated several <code>DecisionTreeClassifier</code>. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.
</div>

In [14]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score

ntrees = 5
max_depth = 3

tree_list = []
ypred_list = []
for i in range(ntrees):
    X_train_i, y_train_i = bootstrap_sample(X_train, y_train)
    indices = np.random.choice(
        np.arange(X.shape[1]), size=int(np.sqrt(X.shape[1])), replace=False
    )
    X_train_i = X_train_i[:, indices]
    treei = DecisionTreeClassifier(max_depth=max_depth)
    treei.fit(X_train_i, y_train_i)
    ypred_list.append(treei.predict(X_test[:, indices]))
    tree_list.append(treei)

ypred, _ = mode(np.array(ypred_list), axis=0, keepdims=False)
print(accuracy_score(y_test, ypred))

1.0


<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own random forest classifier, use a <code>RandomForestClassifier</code> from scikit-learn to fit the above data.
</div>

In [15]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=5)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.9736842105263158


In [16]:
from figures import plot_forest_interactive

plot_forest_interactive()

interactive(children=(IntSlider(value=0, description='max_depth', max=8), Output()), _dom_classes=('widget-int…

In [17]:
from sklearn.ensemble import HistGradientBoostingClassifier